# Вебинар 5. Домашнее задание

Само домашнее задание нахожится в конце ноутбука

---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

# Функции из 1-ого вебинара
import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items

In [2]:
data = pd.read_csv('../data/retail_train.csv')



-----

# Домашнее задание

1. Изучите структуру модуля src
2. Перенесите функции prefilter_items и postfilter_items в модуль src.utils.py
3. Реализуйте функции get_similar_items_recommendation, get_similar_users_recommendation (они разбирались на вебинаре) и переместите в src.utils.py.  
postfilter можно пока оставить пустой (или как-то бороться с товаром 999999) - на Ваше усмотрение.  
В prefilter нужно реализовать закомментированные ограничения.
4. Создайте модуль src.recommenders.py. Напишите код для класса ниже и положите его в src.recommenders.py  
get_similar_items_recommendation и get_similar_users_recommendation - реализуйте их в качетсве методов класса.  
Посчитайте с помощью них рекоменадции и найдите precision@5
5. Проверьте, что все модули корректно импортируются
6. Если вы еще не прочитали [статью](https://habr.com/ru/company/hh/blog/347276/) о рекомендательных системах и поиске в hh.ru, то обязательно прочитайте

**Внимание!** модуль для вашего удобства - дополняйте его теми функциями, которые Вам пригодятся.  
Формальное задание только на prefiletr_items 

In [12]:
def prefilter_items(data_train):
    # Оставим только 5000 самых популярных товаров
    popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
    popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
    top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
    #добавим, чтобы не потерять юзеров
    data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999999 
    
    
    
    # Уберем самые популярные 
    
    # Уберем самые непопулряные 
    
    # Уберем товары, которые не продавались за последние 12 месяцев
    
    # Уберем не интересные для рекоммендаций категории (department)
    
    # Уберем слишком дешевые товары (на них не заработаем). 1 покупка из рассылок стоит 60 руб. 
    
    # Уберем слишком дорогие товары
    

    return data_train

def postfilter_items():
    pass

In [13]:
def get_similar_items_recommendation(user, model, N=5):
    """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
    
    # your_code
    
    return res

def get_similar_users_recommendation(user, model, N=5):
    """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
    
    # your_code
    
    return res

In [14]:
def train_test_split(data,test_size_num,split_column):
    data_train = data[data[split_column] < data[split_column].max() - test_size_num]
    data_test = data[data[split_column] >= data[split_column].max() - test_size_num]
    return data_train, data_test
    

In [18]:
# test_size_weeks = 3
# data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
# data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

# popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
# popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
# top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()


In [19]:
import pandas as pd
import numpy as np

# Для работы с матрицами
from scipy.sparse import csr_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import ItemItemRecommender  # нужен для одного трюка
from implicit.nearest_neighbours import bm25_weight, tfidf_weight


class MainRecommender:
    
    own_recomender_defult_param = {'filter_already_liked_items':False, 
                        'filter_items':False, 
                        "recalculate_user":True}
    
    model_als_defult_param ={'factors':50, 'regularization':15, 'iterations':15, 
                             'num_threads':-1,'calculate_training_loss':False}
    
    def __init__(self, data,data_test=None,split_info=None):
        """ data - dataframe c данными
            data_test - даные для валидации, если нет и есть split_info то создаем
            split_info кортеж с инфрмацией как создать data_test (размер, поле деления) рассматривается только в слуяае отсутвя 
            data_test
        """     
        
        self.data_validation={}
        self.data_validation['status'] = False
        self.user_item_matrix = {'status':False,'matrix':None,'params':None}
        self.own_recommender_is_fit= {'status':False,'params':None}
        self.als_recommender_is_fit= {'status':False,'params':None}
        self.data = data.copy()
        self.full_data_train = data.copy() #Оставим полный объем данный , если нужно будет предсказывать по полному объему данных
        self.data_train = data.copy()
        if data_test is not None:
            self.data_test = data_test.copy()
        else:
            self.data_test = None
            if split_info:
                self.data_train,self.data_test = self.train_test_split(test_size_num = split_info[0],split_column =split_info[1])
        if  self.data_test is not None:
            self.data_validation['data'] = self.get_validation_data()
            self.data_validation['status'] = True


 
    def prefiltr_1(self,my_data):
        # Оставим только 5000 самых популярных товаров
        popularity = my_data.groupby('item_id')['quantity'].count().reset_index()
        popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)
        top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()
        my_data.loc[~my_data['item_id'].isin(top_5000), 'item_id'] = 999999 
        return my_data

  
    def train_test_split(self,test_size_num,split_column):
        data_train = self.data[self.data[split_column] < self.data[split_column].max() - test_size_num]
        data_test = self.data[self.data[split_column] >= self.data[split_column].max() - test_size_num]
        return data_train, data_test
    
    
   
    def get_validation_data(self):
        result = self.data_test.groupby('user_id')['item_id'].unique().reset_index()
        result['train'] = result['user_id'].map(self.data_train.groupby('user_id')['item_id'].unique())
        result['full_train'] = result['user_id'].map(self.full_data_train.groupby('user_id')['item_id'].unique())
        result.rename(columns={'item_id':'test'},inplace=True)
        return result

 
    def prepare_matrix(self,agg_column,full=None,filtr=None):
        my_data = self.data_train
        if full:
            my_data = self.full_data_train
        if  filtr:
            for i in filtr:
                prefiltr = 'self.prefiltr_'+str(i)+'(my_data)'
                my_data = eval(prefiltr)
            
        user_item_matrix = pd.pivot_table(my_data, 
                              index='user_id', columns='item_id', 
                              values=agg_column[0], 
                              aggfunc=agg_column[1], 
                              fill_value=0
                             )
        
        user_item_matrix = user_item_matrix.astype(float) 
        self.prepare_dicts(user_item_matrix)
        

        return user_item_matrix
            


    def prepare_dicts(self,user_item_matrix):
        """Подготавливает вспомогательные словари"""
        
        userids = user_item_matrix.index.values
        itemids = user_item_matrix.columns.values

        matrix_userids = np.arange(len(userids))
        matrix_itemids = np.arange(len(itemids))

        self.id_to_itemid = dict(zip(matrix_itemids, itemids))
        self.id_to_userid = dict(zip(matrix_userids, userids))

        self.itemid_to_id = dict(zip(itemids, matrix_itemids))
        self.userid_to_id = dict(zip(userids, matrix_userids))
        
        return  self.id_to_itemid,  self.id_to_userid,  self.itemid_to_id,  self.userid_to_id
    
    
     
    def make_data(self,agg_column,filtr=None,full =False):
        self.full = full
        uim = self.prepare_matrix(agg_column=agg_column,full=full,filtr=filtr)
        uim_w = uim.copy()
        self.user_item_matrix['uim_matrix_w'] = csr_matrix(uim_w).tocsr()
        uim[uim>0]=1
        self.user_item_matrix['uim_matrix'] = csr_matrix(uim).tocsr()
        
        self.user_item_matrix['ium_matrix_w_tfidf'] = tfidf_weight(csr_matrix(uim_w.T).tocsr())
        self.user_item_matrix['ium_matrix_tfidf'] = tfidf_weight(csr_matrix(uim.T).tocsr())
        self.user_item_matrix['ium_matrix_w_bm25'] = bm25_weight(csr_matrix(uim_w.T).tocsr())
        self.user_item_matrix['ium_matrix_bm25'] = bm25_weight(csr_matrix(uim.T).tocsr())

        self.user_item_matrix['status'] = True
        self.user_item_matrix['params'] = {'agg_column':agg_column,'filtr':filtr,'full':full}
        return self.user_item_matrix
            
        
    def precision_at_k(x, k=5):
    
        bought_list = np.array(x['test'])
        recommended_list = np.array(x['predict'])[:k]
        if len(recommended_list) == 0:
            return 0


        flags = np.isin(bought_list, recommended_list)
        precision = flags.sum() / len(recommended_list)


        return precision
        
        
    
    def fit_own_recommender(self,weighting=False):
        """Обучает модель, которая рекомендует товары, среди товаров, купленных юзером"""
        
        assert self.user_item_matrix['status'], 'необходимо сначала выполнить метод make_data(self,agg_column,filtr=None,weighting=None,full =False)'
        ium = self.user_item_matrix['uim_matrix'].T
        if weighting:
            assert (weighting == 'tf_idf' or weighting == 'bm25'), 'необходимо указать weighting: tf_idf или bm25 или None'
            if  weighting == 'tf_idf':
                ium = self.user_item_matrix['ium_matrix_tfidf']
            else:
                ium = self.user_item_matrix['ium_matrix_bm25']   
        self.own_recommender = ItemItemRecommender(K=1, num_threads=-1)
        self.own_recommender.fit(ium)      
        self.own_recommender_is_fit['status'] =True
        self.own_recommender_is_fit['params'] ={'model':'ItemItemRecommender(K=1, num_threads=-1)','weighting':weighting}
        self.own_recommender_is_fit['ium']=ium
        
        return self.own_recommender
    
    
    def predict_own_recommender(self,users,N=5,params=own_recomender_defult_param):
        
        param = params.copy()
        assert self.own_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_own_recommender()'
        assert type(users) == list, 'users - должен быть списком'
        uim = self.user_item_matrix['uim_matrix']
        param['user_items'] = uim
        param['N'] = N
        answer = pd.DataFrame()
        answer['user_id']=users
        if param['filter_items']:
            param['filter_items']=[self.itemid_to_id[i] for i in params['filter_items']]
        rec=[]
        for user in users:
            param['userid'] = self.userid_to_id[user]
            rec.append( [self.id_to_itemid[i[0]] for i in self.own_recommender.recommend(**param)])
        answer['result']  = rec
        return answer

    
    
    def validation_own_recommender(self,metric=precision_at_k,N=5,params=own_recomender_defult_param):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.own_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_own_recommender()'
        df = self.data_validation['data']
        users = df['user_id'].to_list()
        predict = self.predict_own_recommender(users = users,N=N,params=params)
        df['predict'] = predict['result']
        
        return df.apply(metric,axis=1).mean()
            
        
  
    def fit_als(self, params = model_als_defult_param,weighting=False):
        """Обучает ALS"""
        
        assert self.user_item_matrix['status'], 'необходимо сначала выполнить метод make_data(self,agg_column,filtr=None,weighting=None,full =False)'
        ium = self.user_item_matrix['uim_matrix_w'].T
        if weighting:
            assert (weighting == 'tf_idf' or weighting == 'bm25'), 'необходимо указать weighting: tf_idf или bm25 или None'
            if  weighting == 'tf_idf':
                ium = self.user_item_matrix['ium_matrix_w_tfidf']
            else:
                ium = self.user_item_matrix['ium_matrix_w_bm25']
        
        self.model_als = AlternatingLeastSquares(**params)
        self.model_als.fit(ium)
        self.als_recommender_is_fit['status'] = True
        self.als_recommender_is_fit['params'] = {'model':params,'weighting':weighting}
        self.als_recommender_is_fit['ium'] = ium
        
        return self.model_als
    
    
    def predict_als(self,users,N=5,params=own_recomender_defult_param):
        
        param = params.copy()
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        assert type(users) == list, 'users - должен быть списком'
        uim = self.user_item_matrix['uim_matrix_w']
        param['user_items'] = uim
        param['N'] = N
        answer = pd.DataFrame()
        answer['user_id']=users
        if param['filter_items']:
            param['filter_items']=[self.itemid_to_id[i] for i in params['filter_items']]
        rec=[]
        for user in users:
            param['userid'] = self.userid_to_id[user]
            rec.append( [self.id_to_itemid[i[0]] for i in self.model_als.recommend(**param)])
        answer['result']  = rec
        return answer
    
    
    def validation_als_recommender(self,metric=precision_at_k,N=5,params=own_recomender_defult_param):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.predict_als(users = users,N=N,params=params)
        df['predict'] = predict['result']

        return df.apply(metric,axis=1).mean()  
    
    
    def get_recs(self,user,popularity):
        result = []
        for item in popularity[popularity['user_id']==user]['item_id'].to_list():
            recs_ = self.model_als.similar_items(self.itemid_to_id[item], N=2)
            recs = [self.id_to_itemid[i[0]] for i in recs_]
            if 999999 in recs:
                recs.remove(999999)
            result.append(recs[0])    
        return  result      


    def get_similar_items_recommendation(self, users, N=5):
        """Рекомендуем товары, похожие на топ-N купленных юзером товаров"""
        assert  self.als_recommender_is_fit['status'],'Модель als не обучена, используйте fit_als()'
        assert  type(users)==list,'параметр users должен быть list'
        if self.full:
            my_data = self.full_data_train.copy()
        else:
            my_data = self.data_train.copy()    
        my_data = my_data[my_data['user_id'].isin(users)]    
        popularity = my_data.groupby(['user_id', 'item_id'])['quantity'].count().reset_index()
        popularity.sort_values('quantity', ascending=False, inplace=True)
        popularity = popularity[popularity['item_id'] != 999999]
        popularity =popularity.groupby('user_id').head(N)
        popularity.sort_values(['user_id','quantity'], ascending=False, inplace=True)
        result = pd.DataFrame()
        result['user_id'] = users
        result['similar_recommendation'] = result['user_id'].apply(lambda x: self.get_recs(user = x,popularity = popularity))

        return result
    
    
    def validation_similar_items_recommendation(self,metric=precision_at_k,N=5):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.get_similar_items_recommendation(users = users,N=N)
        df['predict'] = predict['similar_recommendation']

        return df.apply(metric,axis=1).mean() 
    
    
    
    def get_recs_user(self,user,N):
        recs_ = self.model_als.similar_users(self.userid_to_id[user], N=N+1)
        recs = [self.id_to_itemid[i[0]] for i in recs_]
        if 999999 in recs:
            recs.remove(999999)
        
        return  recs[:N]  
    
    def get_similar_users_recommendation(self, users, N=5):
        """Рекомендуем топ-N товаров, среди купленных похожими юзерами"""
        assert  self.als_recommender_is_fit['status'],'Модель als не обучена, используйте fit_als()'
        assert  type(users)==list,'параметр users должен быть list'
        if self.full:
            my_data = self.full_data_train.copy()
        else:
            my_data = self.data_train.copy() 
        result = pd.DataFrame()    
        result['user_id'] = users
        result['similar_recommendation'] = result['user_id'].apply(lambda x: self.get_recs_user(user=x,N=N))

        return result    
            
    def validation_similar_users_recommendation(self,metric=precision_at_k,N=5):
        assert self.data_validation['status'], 'тестовые данные не созданы'
        assert self.als_recommender_is_fit['status'], 'необходимо сначала выполнить метод fit_als()'
        df = self.data_validation['data'].copy()
        users = df['user_id'].to_list()
        predict = self.get_similar_users_recommendation(users = users,N=N)
        df['predict'] = predict['similar_recommendation']

        return df.apply(metric,axis=1).mean()     
    


Работа класса

In [20]:
%%time
# создаем объект класса. data  - полный набор данных,  
# split_info - кортеж, как разбить данные на трайн и тест, если нужно. 
# Если есть тестовые данные их можно передать в парамаетр data_test 
# week_no - поле по которому делим трейн и тест, 3 значение отсечения

my_recomender = MainRecommender(data=data,split_info=(3,'week_no'))

Wall time: 818 ms


In [21]:
%%time
# создадим все маттрицы для работы
# agg_column - поле агрегации и тип агрегации для сводной таблицы
# filtr - указываем каким префильтрам подвергнуть матрицу при создании.Это список. Пока имеется только префильтр _1 отбор 5000 
# самых популярных товаров. Замена остальных на 999999
# full - если нужно сдлеать матицу из полного набьора данных, если Fals то делает из тренировочного,
# полученого делением полного на трайн и тест
my_recomender.make_data(agg_column=('quantity','count'),filtr=[1],full =False)

C:\Users\lqw\AppData\Roaming\Python\Python38\site-packages\pandas\core\indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


Wall time: 2.84 s


{'status': True,
 'matrix': None,
 'params': {'agg_column': ('quantity', 'count'), 'filtr': [1], 'full': False},
 'uim_matrix_w': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'uim_matrix': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'ium_matrix_w_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_w_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>}

In [22]:
# Посмотрим что теперь имеется в классе
my_recomender.data # - полный наьор данных
my_recomender.data_test #- тестовый набор данных
my_recomender.data_train #- набор данных для обучения
my_recomender.userid_to_id
my_recomender.itemid_to_id
my_recomender.id_to_itemid
my_recomender.id_to_userid
None


In [23]:
# Набор данных для валидации на тесте , трейне
my_recomender.data_validation['data']

,user_id,test,train,full_train
0,1,"[821867, 834484, 856942, 865456, 889248, 90795...","[825123, 831447, 840361, 845307, 852014, 85498...","[825123, 831447, 840361, 845307, 852014, 85498..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963...","[866211, 878996, 882830, 904360, 921345, 93194...","[866211, 878996, 882830, 904360, 921345, 93194..."
2,6,"[920308, 926804, 946489, 1006718, 1017061, 107...","[840361, 851494, 851819, 851903, 863447, 87623...","[840361, 851494, 851819, 851903, 863447, 87623..."
3,7,"[840386, 889774, 898068, 909714, 929067, 95347...","[865569, 886703, 889731, 893400, 995436, 10205...","[865569, 886703, 889731, 893400, 995436, 10205..."
4,8,"[835098, 872137, 910439, 924610, 992977, 10412...","[824550, 824555, 826578, 835576, 841220, 84798...","[824550, 824555, 826578, 835576, 841220, 84798..."
...,...,...,...,...
2037,2496,[6534178],"[840361, 852159, 871756, 886703, 899624, 91612...","[840361, 852159, 871756, 886703, 899624, 91612..."
2038,2497,"[1016709, 9835695, 1132298, 16809501, 845294, ...","[838220, 1037840, 1052294, 5569230, 8090537, 1...","[838220, 1037840, 1052294, 5569230, 8090537, 1..."
2039,2498,"[15716530, 834484, 901776, 914190, 958382, 972...","[824555, 835576, 901776, 904023, 911215, 91749...","[824555, 835576, 901776, 904023, 911215, 91749..."
2040,2499,"[867188, 877580, 902396, 914190, 951590, 95813...","[838186, 853197, 864143, 883665, 932949, 93383...","[838186, 853197, 864143, 883665, 932949, 93383..."


In [24]:
# 4 матрицы в наборе а так же указаны параметры агрегации и фильтрации
my_recomender.user_item_matrix

{'status': True,
 'matrix': None,
 'params': {'agg_column': ('quantity', 'count'), 'filtr': [1], 'full': False},
 'uim_matrix_w': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'uim_matrix': <2499x5001 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in Compressed Sparse Row format>,
 'ium_matrix_w_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_tfidf': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_w_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>,
 'ium_matrix_bm25': <5001x2499 sparse matrix of type '<class 'numpy.float64'>'
 	with 679365 stored elements in COOrdinate format>}

In [25]:
# Обучим own_recommender без взешивания
my_recomender.fit_own_recommender()

In [26]:
# параметры предсказаний
params = {'filter_already_liked_items':False, 
                        'filter_items':[999999], 
                        "recalculate_user":True}

In [27]:
# сделаем 5 предсказания для юзеров 1 и 2
my_recomender.predict_own_recommender([1,2],5,params=params)

,user_id,result
0,1,"[1082185, 1029743, 1004906, 5978656, 1081177]"
1,2,"[1082185, 1098066, 1127831, 1004906, 1081177]"


In [28]:
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

0.19711067580802874

In [29]:
# Обучим own_recommender tf_idf
my_recomender.fit_own_recommender(weighting ='tf_idf')

In [30]:
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

0.21101860920665738

In [31]:
# Обучим own_recommender bm25
my_recomender.fit_own_recommender(weighting ='bm25')

In [32]:
%%time
# посмотрим  валидацию на тесте
my_recomender.validation_own_recommender(params=params)

Wall time: 134 ms


0.06520568070519137

In [33]:
# Обучим модель als
param_als = {'factors':1100, 'regularization':35, 'iterations':20, 
                             'num_threads':-1,'calculate_training_loss':True}


In [34]:
%%time
my_recomender.fit_als(params=param_als,weighting ='tf_idf')


Wall time: 13.2 s


In [35]:
%%time
# предикт als
my_recomender.predict_als([1,2,3,4,5,6,7,8,9,10],5,params=params)

Wall time: 7.7 s


,user_id,result
0,1,"[995242, 1082185, 9527290, 840361, 940947]"
1,2,"[1133018, 1106523, 5569230, 1082185, 1098066]"
2,3,"[1106523, 951590, 1133018, 1092026, 910032]"
3,4,"[883932, 962229, 883404, 1075368, 5569230]"
4,5,"[1082185, 1126899, 1029743, 916122, 5568378]"
5,6,"[1082185, 878996, 1024306, 834484, 840361]"
6,7,"[1082185, 1106523, 1122358, 1127831, 1133018]"
7,8,"[1082185, 826249, 1133018, 1029743, 1004906]"
8,9,"[1029743, 1082185, 995242, 1098066, 826249]"
9,10,"[1106523, 1082185, 1133018, 1113588, 1005186]"


In [36]:
%%time
# oценка валидации als на тесте precision@5 tf_idf
my_recomender.validation_als_recommender(params=params)

Wall time: 32min 39s


0.3028403525954943

In [37]:
# обучим без взвешивания
my_recomender.fit_als(params=param_als)

In [38]:
%%time
# oценка валидации на тесте precision@5 без взвешивания
my_recomender.validation_als_recommender(params=params)

Wall time: 32min 39s


0.2787463271302631

In [39]:
# обучим без bm25
my_recomender.fit_als(params=param_als,weighting ='bm25')

In [40]:
%%time
# oценка валидации на тесте precision@5  bm52
my_recomender.validation_als_recommender(params=params)

Wall time: 32min 8s


0.3441723800195895

In [41]:
%%time
# item_item рекомендации
my_recomender.get_similar_items_recommendation( [1,2,3,4,5,6,7,8,9,10], N=5)


Wall time: 97.1 ms


,user_id,similar_recommendation
0,1,"[856942, 1082185, 995242, 940947, 9527290]"
1,2,"[1133018, 1106523, 8090521, 5569230, 916122]"
2,3,"[1092026, 1053690, 951590, 998206, 910032]"
3,4,"[883932, 891423, 962229, 910109, 1052294]"
4,5,"[1126899, 874972, 5995609, 5568378, 1123022]"
5,6,"[1082185, 1119051, 840361, 1037863, 863447]"
6,7,"[1082185, 1122358, 6944571, 1022003, 828867]"
7,8,"[1082185, 1133018, 969932, 1005186, 5569230]"
8,9,"[1029743, 862799, 826249, 1098066, 1056005]"
9,10,"[1106523, 945779, 1113588, 883003, 892008]"


In [42]:
%%time
# item_item рекомендации валидация на тесте precision@5 'bm25'
my_recomender.validation_similar_items_recommendation()

Wall time: 5.06 s


0.4002693437806084

In [43]:
%%time
# user_item рекомендации
my_recomender.get_similar_users_recommendation( users=[1,2,3,4,5,6,7,8,9,10], N=5)

Wall time: 37 ms


,user_id,similar_recommendation
0,1,"[397896, 924786, 987734, 863030, 958729]"
1,2,"[818980, 853235, 1000488, 978974, 983316]"
2,3,"[818981, 962229, 976300, 865456, 1005935]"
3,4,"[819063, 866950, 952981, 862866, 870428]"
4,5,"[819255, 1012701, 923967, 900491, 959219]"
5,6,"[819304, 896617, 942088, 918684, 873916]"
6,7,"[819308, 987218, 835576, 968932, 860037]"
7,8,"[819330, 862152, 990762, 982955, 825731]"
8,9,"[819518, 888431, 995134, 833351, 993044]"
9,10,"[819594, 981760, 879445, 914077, 840609]"


In [44]:
%%time
# user_item валидация на тесте precision@5 'bm25'
my_recomender.validation_similar_users_recommendation()

Wall time: 376 ms


0.005386875612144953

In [47]:
# обучим als без взвешивания
my_recomender.fit_als(params=param_als)

In [48]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешивания
my_recomender.validation_similar_items_recommendation()

Wall time: 5.71 s


0.4002693437806084

In [49]:
%%time
# user_item валидация на тесте precision@5 без взвешиания
my_recomender.validation_similar_users_recommendation()

Wall time: 390 ms


0.006758080313418212

In [50]:
%%time
# item_item рекомендации валидация на тесте precision@5 tf_idf
my_recomender.fit_als(params=param_als,weighting ='tf_idf')


Wall time: 13.4 s


In [51]:
%%time
# item_item рекомендации валидация на тесте precision@5 без взвешивания
my_recomender.validation_similar_items_recommendation()

Wall time: 6.07 s


0.4002693437806084

In [52]:
%%time
# user_item валидация на тесте precision@5 без взвешиания
my_recomender.validation_similar_users_recommendation()

Wall time: 386 ms


0.006072477962781582

----

Проверка, что все работает

In [45]:
from src.metrics import precision_at_k, recall_at_k
from src.utils import prefilter_items
from src.recommenders import MainRecommender



ModuleNotFoundError: No module named 'src.recommenders'